In [1]:
import os
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()
import pathlib as Path

In [2]:
# storing alpaca key and secret key in variables 
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

api = tradeapi.REST(alpaca_api_key,alpaca_secret_key,api_version='V2')

In [3]:
# set time frame
timeframe = '1D'

# start and 'end' date for covid stock market 
start_date_covid = pd.Timestamp('2020-01-01',tz='America/New_York').isoformat()
end_date_covid = pd.Timestamp('2020-11-20',tz='America/New_York').isoformat()

# start and end date for 2008 recession 
start_date_2008 = pd.Timestamp('2007-07-01',tz='America/New_York').isoformat()
end_date_2008 = pd.Timestamp('2009-07-01',tz='America/New_York').isoformat()

# set tickers (per industry)
energy_tickers = ['XOM','BP','COP','CVX', 'TOT']
# financeind_tickers
# techind_tickers etc

# get stock data from covid
energy_covid_ticker = api.get_barset(energy_tickers,timeframe,start=start_date_covid,end = end_date_covid).df

# get stock data from 2008 recession
energy_ticker2008 = api.get_barset(energy_tickers,timeframe,start=start_date_2008,end = end_date_2008).df

In [4]:
# create new covid and 2008 dataframe to store ckosing prices of each stock

energy_covid_closingprices = pd.DataFrame()

# get closing prices for all tickers
for ticker in energy_tickers:
    energy_covid_closingprices[ticker]=energy_covid_ticker[ticker]['close']
    
# drop time component on index
energy_covid_closingprices.index = energy_covid_closingprices.index.date
    
energy_covid_closingprices.head()

,XOM,BP,COP,CVX,TOT
2020-01-02,70.900,38.14,65.470,121.44,55.28
2020-01-03,70.320,38.84,65.710,121.01,55.47
2020-01-06,70.870,39.85,66.480,120.61,56.53
2020-01-07,70.280,39.44,66.480,119.04,56.19
2020-01-08,69.235,38.83,64.935,117.69,55.85


In [5]:
energy_closingprices2008 = pd.DataFrame()

# get closing prices for all tickers
for ticker in energy_tickers:
    energy_closingprices2008[ticker]=energy_ticker2008[ticker]['close']
    
# drop time component on index
energy_closingprices2008.index = energy_closingprices2008.index.date
    
energy_covid_closingprices.head()

,XOM,BP,COP,CVX,TOT
2020-01-02,70.900,38.14,65.470,121.44,55.28
2020-01-03,70.320,38.84,65.710,121.01,55.47
2020-01-06,70.870,39.85,66.480,120.61,56.53
2020-01-07,70.280,39.44,66.480,119.04,56.19
2020-01-08,69.235,38.83,64.935,117.69,55.85


In [20]:
energy_covid_dr = energy_covid_closingprices.pct_change().dropna()
enerygy_returns2008 = energy_closingprices2008.pct_change().dropna()

### Covid Weighting

In [7]:
#Name of stocks in portfolio
print(energy_covid_closingprices.columns)

energy_covid_weights = np.full((1, len(energy_covid_closingprices.columns)), 1/len(energy_covid_closingprices.columns))[0]
print("Weights")
print(energy_covid_weights)

#expect returns
print('Expected returns')
energy_covid_exp_return = np.sum((energy_covid_dr.mean()*energy_covid_weights)*252)
print(energy_covid_exp_return)

#expected STD/volatility
print('Expected volatility')
energy_covid_exp_volat = np.sqrt(np.dot(energy_covid_weights.T, np.dot(energy_covid_dr.cov()*252, energy_covid_weights)))
print(energy_covid_exp_volat)

#Shape ratio
print('Shape ratio')
energy_covid_sharpe = energy_covid_exp_return/energy_covid_exp_volat
print(energy_covid_sharpe)

Index(['XOM', 'BP', 'COP', 'CVX', 'TOT'], dtype='object')
Weights
[0.2 0.2 0.2 0.2 0.2]
Expected returns
-0.34866491699929814
Expected volatility
0.5889978255446008
Shape ratio
-0.5919629952401176


In [9]:
# running 100k simulations on portfolio
number_ports = 100000
energy_covid_all_weights = np.zeros((number_ports, len(energy_covid_closingprices.columns)))
energy_covid_returns_array = np.zeros(number_ports)
energy_covid_volatility_array = np.zeros(number_ports)
energy_covid_sharpe_array = np.zeros(number_ports)

np.random.seed(5)

for index in range(number_ports):
  #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(energy_covid_weights)

  #save weights
    energy_covid_all_weights[index,:] = energy_covid_weights

  #expected returns
    energy_covid_returns_array[index] = np.sum((energy_covid_dr.mean()*weights)*252)

  #expected volatiltiy
    energy_covid_volatility_array[index] = np.sqrt(np.dot(weights.T, np.dot(energy_covid_dr.cov()*252, weights)))

  #sharpe ratio
    energy_covid_sharpe_array[index] = energy_covid_returns_array[index]/energy_covid_volatility_array[index]

In [11]:
#find highest sharpe ratio
energy_covid_sharpe_array.max()

-0.28559893647575174

In [16]:
#find shape weights by index
energy_covid_index_max_shape = energy_covid_sharpe_array.argmax()
energy_covid_index_max_shape

8950

In [26]:
#look at the weights from the index
energy_covid_weights_array = energy_covid_all_weights[energy_covid_index_max_shape,:]
energy_covid_weights_array

array([0.08202324, 0.32172289, 0.07637971, 0.33941334, 0.18046081])

### 2008 Weighting

In [21]:
#Name of stocks in portfolio
print(energy_closingprices2008.columns)

energy_2008_weights = np.full((1, len(energy_closingprices2008.columns)), 1/len(energy_closingprices2008.columns))[0]
print("Weights")
print(energy_2008_weights)

#expect returns
print('Expected returns')
energy_2008_exp_return = np.sum((enerygy_returns2008.mean()*energy_2008_weights)*252)
print(energy_2008_exp_return)

#expected STD/volatility
print('Expected volatility')
energy_2008_exp_volat = np.sqrt(np.dot(energy_2008_weights.T, np.dot(enerygy_returns2008.cov()*252, energy_2008_weights)))
print(energy_2008_exp_volat)

#Shape ratio
print('Shape ratio')
energy_2008_sharpe = energy_2008_exp_return/energy_2008_exp_volat
print(energy_2008_sharpe)

Index(['XOM', 'BP', 'COP', 'CVX', 'TOT'], dtype='object')
Weights
[0.2 0.2 0.2 0.2 0.2]
Expected returns
1.3460929828034986
Expected volatility
0.9213353784203188
Shape ratio
1.4610238728826963


In [22]:
# running 100k simulations on portfolio
number_ports = 100000
energy_2008_all_weights = np.zeros((number_ports, len(energy_closingprices2008.columns)))
energy_2008_returns_array = np.zeros(number_ports)
energy_2008_volatility_array = np.zeros(number_ports)
energy_2008_sharpe_array = np.zeros(number_ports)

np.random.seed(5)

for index in range(number_ports):
  #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(energy_2008_weights)

  #save weights
    energy_2008_all_weights[index,:] = energy_2008_weights

  #expected returns
    energy_2008_returns_array[index] = np.sum((enerygy_returns2008.mean()*weights)*252)

  #expected volatiltiy
    energy_2008_volatility_array[index] = np.sqrt(np.dot(weights.T, np.dot(enerygy_returns2008.cov()*252, weights)))

  #sharpe ratio
    energy_2008_sharpe_array[index] = energy_2008_returns_array[index]/energy_2008_volatility_array[index]

In [23]:
#find highest sharpe ratio
energy_2008_sharpe_array.max()

1.7697987406084699

In [25]:
#find shape weights by index
energy_2008_index_max_shape = energy_2008_sharpe_array.argmax()
energy_2008_index_max_shape

62505

In [27]:
#look at the weights from the index
energy_2008_weights_array = energy_2008_all_weights[energy_2008_index_max_shape,:]
energy_covid_weights_array

array([0.08202324, 0.32172289, 0.07637971, 0.33941334, 0.18046081])